In [108]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import torch
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
import torchtuples as tt
from torchtuples import Model
from sklearn.model_selection import train_test_split

from dqAux import dqNetSparse,checkLoss,checkErrorMean,getSESingle

In [109]:
df  = pd.read_csv("./data/concrete.csv")
df_train,df_test = train_test_split(df,test_size=0.2)

In [110]:
df.head(5)

,blst,flh,sup,cag,fag,age,inX,outY
0,0.0,0.0,2.5,1040.0,676.0,28,0.300000,4.381901
1,0.0,0.0,2.5,1055.0,676.0,28,0.300000,4.125358
2,142.5,0.0,0.0,932.0,594.0,270,0.685714,3.695607
3,142.5,0.0,0.0,932.0,594.0,365,0.685714,3.714791
4,132.4,0.0,0.0,978.4,825.5,360,0.966767,3.790984


In [111]:
# hyperparameters
batch_size = 128
nodes = [5,128] # number of network layers and nodes per layer
lr = 0.001
epochs = 500
verbose = False
sparseRatio = 0.5

In [112]:
df_train_lin = df_train
df_val = df_train.sample(frac=0.1)
df_train = df_train.drop(df_val.index)

In [113]:
# standardize covariates
cols_standardize = ['inX','blst','flh','sup','cag','fag', 'age']
cols_leave = []

standardize = [([col], StandardScaler()) for col in cols_standardize]
leave = [(col, None) for col in cols_leave]
x_mapper = DataFrameMapper(standardize + leave)

x_train_nonpar = torch.tensor(x_mapper.fit_transform(df_train).astype('float32'))
x_val_nonpar = torch.tensor(x_mapper.transform(df_val).astype('float32'))
x_test_nonpar = torch.tensor(x_mapper.transform(df_test).astype('float32'))

x_train_lin = torch.tensor(df_train['inX'].values).view(len(df_train),1).type(torch.float32)
x_test_lin = torch.tensor(df_test['inX'].values).view(len(df_test),1).type(torch.float32)
x_val_lin = torch.tensor(df_val['inX'].values).view(len(df_val),1).type(torch.float32)

y_train = torch.tensor(df_train['outY'].values).view(len(df_train),1).type(torch.float32)
y_test = torch.tensor(df_test['outY'].values).view(len(df_test),1).type(torch.float32)
y_val = torch.tensor(df_val['outY'].values).view(len(df_val),1).type(torch.float32)

x_train = (x_train_lin,x_train_nonpar)
x_test = (x_test_lin,x_test_nonpar)
x_val = (x_val_lin,x_val_nonpar)
val_data = (x_val,y_val)

In [114]:
dim_lin,dim_nonpar=1,x_val_nonpar.shape[1]
tau_seq = [(i+1)/50 for i in range(49)]# quantile interval [0.02,0.98]
resTab = np.zeros((len(tau_seq),5))
resTabLin = np.zeros((len(tau_seq),5))
i=0

In [115]:
# training model with different quantile levels
for tau in tau_seq:
    # linear quantile regression (LQR)
    model_lin = smf.quantreg("outY~inX+blst+flh+sup+cag+fag+age",df_train_lin).fit(q=tau)
    paramsLin = model_lin.params[1]
    model_lin.bse
    model_lin.conf_int().loc['inX']
    # prediction for LQR
    pred_lin = model_lin.predict(df_test).values
    predErrLin = checkErrorMean(pred_lin.reshape(len(pred_lin),1),y_test.numpy(),tau=tau)
    
    # training Partially Linear Quantile Regression (DPLQR)
    coef_init_weight=torch.tensor(paramsLin,dtype=y_train.dtype)
    loss = checkLoss(tau=tau)
    model = Model(dqNetSparse(dim_lin,dim_nonpar,coef_init_weight,nodes,sparseRatio),loss)
    model.optimizer.set_lr(lr)
    callbacks = [tt.callbacks.EarlyStopping()]
    model.fit(x_train,y_train,batch_size,epochs,callbacks,verbose,val_data=val_data, val_batch_size=batch_size)
    weights = list(model.net.parameters())
    
    # calculate standard error
    x_resd = (torch.cat((x_train[0],x_val[0]),dim=0),torch.cat((x_train[1],x_val[1]),dim=0))
    resdErr = (torch.cat((y_train,y_val),dim=0) - model.predict(x_resd)).numpy()

    callbacks = [tt.callbacks.EarlyStopping()]
    # obtain asymptotic covariance matrix
    se = getSESingle(x_train,x_val,resdErr,tau,nodes,batch_size,lr,epochs, callbacks,verbose,logic=True)
    
    # prediction error (w.r.t. check loss) on test set
    preds = model.predict(x_test)
    check_loss_tau = checkErrorMean(preds.numpy(),y_test.numpy(),tau=tau)
    deepCoefEst=weights[0].detach().numpy()[0][0]
    
    intvLin = (model_lin.conf_int().loc['inX'][0],model_lin.conf_int().loc['inX'][1])
    s1=[tau,deepCoefEst,deepCoefEst-1.96*se[0],deepCoefEst+1.96*se[0],check_loss_tau]
    s2=[tau,paramsLin,intvLin[0],intvLin[1],predErrLin]
    
    resTab[i,:] = s1
    resTabLin[i,:] = s2
    i=i+1
    
resDeep = pd.DataFrame(resTab)
resDeep.columns = ['tau','estCoef','lowerCI','upperCI','CL']
resLin = pd.DataFrame(resTabLin)
resLin.columns = ['tau','estCoef','lowerCI','upperCI','CL']

/Users/qxzhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:190: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/qxzhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:190: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/qxzhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:190: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/qxzhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:190: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


In [116]:
# confidence intervals area via Riemann sum
delta = tau_seq[1]-tau_seq[0]
CIArea = ((resDeep['upperCI'] - resDeep['lowerCI'])*delta).sum()
CIAreaLin = ((resLin['upperCI'] - resLin['lowerCI'])*delta).sum()
print("95% CI Area under DPLQR:",CIArea)
print("95% CI Area under LQR:",CIAreaLin)

95% CI Area under DPLQR: 0.2223621286694169
95% CI Area under LQR: 0.3364240113753357


In [117]:
# ACL
acl = (resDeep['CL']*delta).sum()
aclLin = (resLin['CL']*delta).sum()
print('ACL under DPLQR:',acl)
print('ACL under LQR:',aclLin)

ACL under DPLQR: 0.058949435222893955
ACL under LQR: 0.09680439909407373


In [118]:
# save for plot
resDeep.to_csv('./data/resDeep.csv')
resLin.to_csv('./data/resLin.csv')

In [119]:
# save train and test data for PLAQR
# which is implemented by R package plaqr
df_train.to_csv("./data/df_train.csv")
df_test.to_csv("./data/df_test.csv")